In [1]:
# Let's make inference on test data here. Should we do all test blocks at once? I think it would make sense to do  on all blocks together since this is a deeplearning model. Maybe we can do both block-wise and all test dataset wise

In [2]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score - can't use this on 3D matrices, only work for 2D
from scipy.stats import pearsonr

2025-06-10 12:20:50.400460: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 12:20:50.437521: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 12:20:50.437546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 12:20:50.438464: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 12:20:50.444778: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# import the trained seq_2_seq model here
seq_2_seq_generic_model = tf.keras.models.load_model("models/seq_2_seq_generic_model.keras")

2025-06-10 12:20:52.260394: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [4]:
# validation data location
validation_data_loc = "seq_2_seq_valid_data"

In [5]:
os.listdir(validation_data_loc)

['block_0204_extracted_input_features.npy',
 'block_0204_extracted_target_features.npy']

In [6]:
# load validation data
validation_features = np.load(os.path.join(validation_data_loc, 'block_0204_extracted_input_features.npy'))

In [7]:
validation_features.shape

(910, 13, 32)

In [8]:
# get the predictions for validation data
preds_for_valid_data = seq_2_seq_generic_model.predict(validation_features)

29/29 [==============================] - 1s 8ms/step


In [9]:
preds_for_valid_data.shape

(910, 7, 32)

In [10]:
# sanity check - check with the mean on script 2
preds_for_valid_data.mean()

0.3711441

In [11]:
# Let's compute the performance metrics for the validaiton data

In [12]:
# load the true validation data
valid_true = np.load(os.path.join(validation_data_loc, 'block_0204_extracted_target_features.npy'))

In [13]:
valid_true.shape

(910, 7, 32)

In [14]:
# get the metrics with tensorflow

# convert the true and predicted matrices into tensors
y_true_tensor_valid = tf.constant(valid_true, dtype = tf.float32)
y_pred_tensor_valid = tf.constant(preds_for_valid_data, dtype = tf.float32)

In [15]:
y_true_tensor_valid.shape, y_pred_tensor_valid.shape

(TensorShape([910, 7, 32]), TensorShape([910, 7, 32]))

In [16]:
# define the MAE metric
mae_metric_valid = tf.keras.metrics.MeanAbsoluteError()

In [17]:
# update the state of the metric
mae_metric_valid.update_state(y_true_tensor_valid, y_pred_tensor_valid)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6370.0>

In [18]:
# Get the result
mae_valid = mae_metric_valid.result().numpy()

In [19]:
mae_valid # This value match the one during training in script 2

0.15943222

In [20]:
# We can also use numpy to get this value directly
np.mean(np.abs(valid_true - preds_for_valid_data))

0.15943222

In [21]:
# Also get the rmse?

# define the MSE metric
mse_metric_valid = tf.keras.metrics.MeanSquaredError()

In [22]:
# update the state of the metric
mse_metric_valid.update_state(y_true_tensor_valid, y_pred_tensor_valid)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6370.0>

In [23]:
# get the result
mse_valid = mse_metric_valid.result().numpy()
mse_valid

0.1067442

In [24]:
# therefore rmse
rmse_valid = np.sqrt(mse_valid)
rmse_valid

0.32671732

In [25]:
# Do this the numpy way?
np.sqrt(np.mean((valid_true - preds_for_valid_data)**2))

0.32671732